In [ ]:
########9.29
#后续：找落在71个有intron retained的isoform对应的基因区域的res——10.07之前的想法
#10.07除去经选择性剪接分析得到的71个具有IR的iso外，对94个经历了选择性剪接且有明确后果的基因进行igv的详细可视化

In [1]:
##########联动isoform信息与RES
#deEx_iso:经过if usage差异表达得到的587个iso，进一步对dif+p值过滤，得到显著差异表达的125个iso
#其中71个iso具有IR
deEx_iso_IR <- read.csv("/disk1/wenqing/tmp_data/ASD/deEx_isoform_IR.csv")
dim(deEx_iso_IR)
head(deEx_iso_IR)

[1] 71 36

,X,iso_ref,gene_ref,isoform_id,gene_id,condition_1,condition_2,gene_name,gene_biotype,iso_biotype,⋯,IF2,dIF,isoform_switch_q_value,gene_switch_q_value,PTC,codingPotentialValue,codingPotential,domain_identified,IR,switchConsequencesGene
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>,<dbl>,<lgl>,<chr>,<int>,<lgl>
1,117219,isoComp_00000024,geneComp_00000007,ENST00000667950.1_2,A2M-AS1,ASD,Control,A2M-AS1,lncRNA,lncRNA,⋯,0.06853333,0.068533333,5.463666e-01,3.487953e-06,FALSE,0.004551983,FALSE,no,1,TRUE
2,117220,isoComp_00000025,geneComp_00000007,ENST00000670582.1_1,A2M-AS1,ASD,Control,A2M-AS1,lncRNA,lncRNA,⋯,0.00000000,-0.184900000,3.487953e-06,3.487953e-06,FALSE,0.004547452,FALSE,no,1,TRUE
3,67064,isoComp_00005868,geneComp_00001500,ENST00000504484.5_3,APBB2,ASD,Control,APBB2,protein_coding,retained_intron,⋯,0.02260000,0.001433333,9.063016e-01,3.177904e-02,FALSE,0.015046990,FALSE,no,1,TRUE
4,95259,isoComp_00007285,geneComp_00001787,ENST00000361910.9_6,ARMCX6,ASD,Control,ARMCX6,protein_coding,protein_coding,⋯,0.11586667,0.033800000,9.853710e-01,3.302405e-02,FALSE,0.143687731,FALSE,yes,1,TRUE
5,50454,isoComp_00012061,geneComp_00002831,ENST00000333234.5_1,C18orf21,ASD,Control,C18orf21,protein_coding,protein_coding,⋯,0.07400000,0.074000000,5.555719e-01,6.346829e-14,FALSE,0.006724984,FALSE,yes,1,FALSE
6,41918,isoComp_00020293,geneComp_00004590,ENST00000584581.5_1,CNDP2,ASD,Control,CNDP2,protein_coding,retained_intron,⋯,0.27540000,0.087100000,8.303687e-01,9.580512e-19,TRUE,0.224747935,FALSE,yes,2,TRUE


In [2]:
length(unique(deEx_iso_IR$gene_id))
table(deEx_iso_IR$gene_id)

[1] 39


        A2M-AS1           APBB2          ARMCX6        C18orf21           CNDP2 
              2               1               1               1               2 
         COPS7B           CYREN          DCAF11          DCAF17         DNAJC10 
              2               1               4               1               3 
ENSG00000287625 ENSG00000291171         FAM193A         FAM200B          FIGNL1 
              2               1               1               6               1 
        GAS6-DT         GLUD1P3             HGF          HS1BP3           IFIT5 
              2               1               1               1               1 
        KRTCAP3           LERFS       LINC03015      LYPLAL1-DT          MAN2B1 
              1               1               2               1               1 
         MIATNB         OLFML2B           PALB2            PAX6            PCTP 
              1               1               1               4               1 
           PEG3          PN

In [3]:
#exon_region：组成isoform的exon的region
exon_region <- read.csv("/disk1/wenqing/tmp_data/ASD/switchList_exon.csv")


In [7]:
head(exon_region)

,X,seqnames,start,end,width,strand,isoform_id,gene_id,gene_name
,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
1,1,chr1,12010,12057,48,+,ENST00000450305.2_2,DDX11L1,DDX11L1
2,2,chr1,12179,12227,49,+,ENST00000450305.2_2,DDX11L1,DDX11L1
3,3,chr1,12613,12697,85,+,ENST00000450305.2_2,DDX11L1,DDX11L1
4,4,chr1,12975,13052,78,+,ENST00000450305.2_2,DDX11L1,DDX11L1
5,5,chr1,13221,13374,154,+,ENST00000450305.2_2,DDX11L1,DDX11L1
6,6,chr1,13453,13670,218,+,ENST00000450305.2_2,DDX11L1,DDX11L1


In [4]:
#res是否落在可变剪接的供受体位点上
#构建供受体位点上下游5bp的潜在res位点
donor_sites_2 <- paste0(exon_region$seqnames,"_",as.character(as.integer(exon_region$end)+2))
donor_sites_1 <- paste0(exon_region$seqnames,"_",as.character(as.integer(exon_region$end)+1))
donor_sites_3 <- paste0(exon_region$seqnames,"_",as.character(as.integer(exon_region$end)+3))
donor_sites_4 <- paste0(exon_region$seqnames,"_",as.character(as.integer(exon_region$end)+4))
donor_sites_5 <- paste0(exon_region$seqnames,"_",as.character(as.integer(exon_region$end)+5))

acceptor_sites_2 <- paste0(exon_region$seqnames,"_",as.character(as.integer(exon_region$start)-2))
acceptor_sites_1 <- paste0(exon_region$seqnames,"_",as.character(as.integer(exon_region$start)-1))
acceptor_sites_3 <- paste0(exon_region$seqnames,"_",as.character(as.integer(exon_region$start)-3))
acceptor_sites_4 <- paste0(exon_region$seqnames,"_",as.character(as.integer(exon_region$start)-4))
acceptor_sites_5 <- paste0(exon_region$seqnames,"_",as.character(as.integer(exon_region$start)-5))


branch_sites <- paste0(exon_region$seqnames,"_",as.character(as.integer(exon_region$start)-20))

all_possible_sites <- c(donor_sites_1,donor_sites_2,donor_sites_3,donor_sites_4,donor_sites_5,acceptor_sites_1,acceptor_sites_2,acceptor_sites_3,acceptor_sites_4,acceptor_sites_5,branch_sites)

donor_sites_info <- c(rep("donor_ds_1_G",times=length(exon_region$seqnames)),rep("donor_ds_2_T",times=length(exon_region$seqnames)),rep("donor_ds_3_A_G",times=length(exon_region$seqnames)),rep("donor_ds_4_A",times=length(exon_region$seqnames)),rep("donor_ds_5_G",times=length(exon_region$seqnames)))
acceptor_sites_info <- c(rep("acceptor_us_1_G",times=length(exon_region$seqnames)),rep("acceptor_us_2_A",times=length(exon_region$seqnames)),rep("acceptor_us_3_C_T",times=length(exon_region$seqnames)),rep("acceptor_us_4_N",times=length(exon_region$seqnames)),rep("acceptor_us_5_C_T",times=length(exon_region$seqnames)))
branch_sites_info <- rep("branch_site_A",times=length(exon_region$seqnames))

all_sites_info <- c(donor_sites_info,acceptor_sites_info,branch_sites_info)
all_sites <- as.data.frame(cbind(all_possible_sites,all_sites_info))


In [5]:

head(all_sites)
dim(all_sites)
colnames(all_sites) <- c("site","info")
head(all_sites)


,all_possible_sites,all_sites_info
,<chr>,<chr>
1,chr1_12058,donor_ds_1_G
2,chr1_12228,donor_ds_1_G
3,chr1_12698,donor_ds_1_G
4,chr1_13053,donor_ds_1_G
5,chr1_13375,donor_ds_1_G
6,chr1_13671,donor_ds_1_G


[1] 10189652        2

,site,info
,<chr>,<chr>
1,chr1_12058,donor_ds_1_G
2,chr1_12228,donor_ds_1_G
3,chr1_12698,donor_ds_1_G
4,chr1_13053,donor_ds_1_G
5,chr1_13375,donor_ds_1_G
6,chr1_13671,donor_ds_1_G


In [6]:
#4218444个候选位点
uniq_all_sites <- all_sites[!duplicated(all_sites$site),]
dim(uniq_all_sites)

rownames(uniq_all_sites) <- uniq_all_sites$site
#write.table(all_sites,"/disk1/wenqing/tmp_data/ASD/donor_acceptor_sites_us_ds_5bp.txt",quote=F,row.names=F)

[1] 4218444       2

In [7]:
#head(all_possible_sites)
all_res <- read.table("/disk1/wenqing/tmp_data/ASD/all.res.wizSampleInfo")

head(all_res)

all_res$V10 <- paste0(all_res$V1,"_",all_res$V3)
head(all_res)



,V1,V2,V3,V4,V5,V6,V7,V8,V9
,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>
1,chr1,99234,99235,TC,2,+,GTCTTCGTCATAACCG,3:3,17
2,chr1,99298,99299,TC,2,+,CATATGGCACCAGGCT,2:2,17
3,chr1,99312,99313,TC,4,+,GTCTTCGTCATAACCG,6:6,17
4,chr1,99326,99327,TC,2,+,GTCTTCGTCATAACCG,7:7,17
5,chr1,103105,103106,TC,1,+,AACTCCCGTAGTGAAT,1:1,17
6,chr1,103106,103107,TC,1,+,AACTCCCGTAGTGAAT,1:1,17


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10
,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>
1,chr1,99234,99235,TC,2,+,GTCTTCGTCATAACCG,3:3,17,chr1_99235
2,chr1,99298,99299,TC,2,+,CATATGGCACCAGGCT,2:2,17,chr1_99299
3,chr1,99312,99313,TC,4,+,GTCTTCGTCATAACCG,6:6,17,chr1_99313
4,chr1,99326,99327,TC,2,+,GTCTTCGTCATAACCG,7:7,17,chr1_99327
5,chr1,103105,103106,TC,1,+,AACTCCCGTAGTGAAT,1:1,17,chr1_103106
6,chr1,103106,103107,TC,1,+,AACTCCCGTAGTGAAT,1:1,17,chr1_103107


In [8]:
#4739个位点落在上述构建的位置上
resInSpliceSites <- all_res[all_res$V10 %in% all_sites$site,]


In [9]:
dim(resInSpliceSites)

[1] 16596    10

In [10]:

resInSpliceSites$V11 <- "info"


In [11]:
resInSpliceSites$V11 <- uniq_all_sites[resInSpliceSites$V10,]$info


In [12]:

names(table(resInSpliceSites$V11))


[1] "acceptor_us_1_G"   "acceptor_us_2_A"   "acceptor_us_3_C_T"
 [4] "acceptor_us_4_N"   "acceptor_us_5_C_T" "branch_site_A"    
 [7] "donor_ds_1_G"      "donor_ds_2_T"      "donor_ds_3_A_G"   
[10] "donor_ds_4_A"      "donor_ds_5_G"

In [244]:
###########暂时不做这种过滤，可能性过多
head(resInSpliceSites)

#判断res所在位点是否有可能造成AS
judge_AS <- function(x){
    candidate_res_info <- c()
    site_info=x[11]
    res_type=x[4]
    flag1 <- site_info == 'acceptor_us_1_G' & res_type=='G'
    flag2 <- site_info == 'acceptor_us_2_A' & res_type=='G'

    if(site_info == 'acceptor_us_1_G' & res_type=='G'){
        candidate_res_info <- rbind(candidate_res_info,x)

    }
    
}

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>
1373,chr1,1215686,1215687,AG,1,+,GGGCACTAGTGTACCT,1:1,17,chr1_1215687,acceptor_us_4_N
1374,chr1,1215688,1215689,AG,1,-,GACTGCGAGGGCTTCC,1:1,17,chr1_1215689,acceptor_us_2_A
1756,chr1,1580934,1580935,TC,3,+,CGTCAGGTCCGTAGGC,3:3,17,chr1_1580935,acceptor_us_5_C_T
1757,chr1,1580967,1580968,TC,3,+,CTACGTCAGTAGATGT,3:3,17,chr1_1580968,acceptor_us_4_N
6017,chr1,6608487,6608488,TC,3,+,TAGAGCTGTCGGGTCT,3:3,17,chr1_6608488,branch_site_A
6303,chr1,6726654,6726655,TC,2,+,GGCCGATTCAATCTCT,9:9,17,chr1_6726655,acceptor_us_2_A


In [248]:
####将得到的res归到基因上
#写出resInSpliceSites
write.table(resInSpliceSites,"/disk1/wenqing/tmp_data/ASD/resInSpliceSites.txt",quote=F,sep='\t',row.names=F)

In [247]:
head(resInSpliceSites)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11
,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>
1373,chr1,1215686,1215687,AG,1,+,GGGCACTAGTGTACCT,1:1,17,chr1_1215687,acceptor_us_4_N
1374,chr1,1215688,1215689,AG,1,-,GACTGCGAGGGCTTCC,1:1,17,chr1_1215689,acceptor_us_2_A
1756,chr1,1580934,1580935,TC,3,+,CGTCAGGTCCGTAGGC,3:3,17,chr1_1580935,acceptor_us_5_C_T
1757,chr1,1580967,1580968,TC,3,+,CTACGTCAGTAGATGT,3:3,17,chr1_1580968,acceptor_us_4_N
6017,chr1,6608487,6608488,TC,3,+,TAGAGCTGTCGGGTCT,3:3,17,chr1_6608488,branch_site_A
6303,chr1,6726654,6726655,TC,2,+,GGCCGATTCAATCTCT,9:9,17,chr1_6726655,acceptor_us_2_A


In [13]:
#读取resInSpliceSites_WzGeneInfo
resInSpliceSites_WzGeneInfo <- read.table("/disk1/wenqing/tmp_data/ASD/resInSpliceSites_WzGeneInfo.txt")

In [14]:

head(resInSpliceSites_WzGeneInfo)

resInSpliceSites_WzGeneInfo_dedup <- resInSpliceSites_WzGeneInfo[!duplicated(resInSpliceSites_WzGeneInfo$V15),]
#resInSpliceSites_WzGeneInfo_dedup维度：4306个位点落在基因的AS site及其上下游5bp的位置
dim(resInSpliceSites_WzGeneInfo_dedup)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17
,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>
1,chr1,1017197,1051741,C1orf159,;,chr1,1033495,1033496,TC,1,+,CCCATACCAGTTTACG,1:1,18,chr1_1033496,donor_ds_3_A_G,1
2,chr1,1017197,1051741,C1orf159,;,chr1,1033495,1033496,TC,2,+,CGATTGATCTGATACG,3:3,20,chr1_1033496,donor_ds_3_A_G,1
3,chr1,1017197,1051741,C1orf159,;,chr1,1033495,1033496,TC,1,+,GGGTTGCTCGATAGAA,2:2,23,chr1_1033496,donor_ds_3_A_G,1
4,chr1,1210602,1215800,RP5-902P8.10,;,chr1,1215686,1215687,AG,1,+,GGGCACTAGTGTACCT,1:1,17,chr1_1215687,acceptor_us_4_N,1
5,chr1,1210602,1215800,RP5-902P8.10,;,chr1,1215688,1215689,AG,1,-,GACTGCGAGGGCTTCC,1:1,17,chr1_1215689,acceptor_us_2_A,1
6,chr1,1210602,1215800,RP5-902P8.10,;,chr1,1215686,1215687,AG,1,-,ATCCACCGTCACCTAA,1:1,18,chr1_1215687,acceptor_us_4_N,1


[1] 4306   17

In [262]:
names(table(resInSpliceSites$V11))

[1] "acceptor_us_1_G"   "acceptor_us_2_A"   "acceptor_us_3_C_T"
 [4] "acceptor_us_4_N"   "acceptor_us_5_C_T" "branch_site_A"    
 [7] "donor_ds_1_G"      "donor_ds_2_T"      "donor_ds_3_A_G"   
[10] "donor_ds_4_A"      "donor_ds_5_G"

In [16]:
sort(table(resInSpliceSites_WzGeneInfo_dedup$V4),decreasing=T)[1:10]
#2295个候选基因，数量太多，根据res类型及其发生的位点设置过滤条件
length(names(table(resInSpliceSites_WzGeneInfo_dedup$V4)))



     JPX    ZNF83    MARK3 OIP5-AS1 TPT1-AS1   AKNAD1   SUZ12P    CDC27 
      25       13       12       12       12       11       11       10 
    CTSB  FAM153B 
      10       10 

[1] 2295

In [265]:
head(resInSpliceSites_WzGeneInfo_dedup)


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17
,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>
1,chr1,1017197,1051741,C1orf159,;,chr1,1033495,1033496,TC,1,+,CCCATACCAGTTTACG,1:1,18,chr1_1033496,donor_ds_3_A_G,1
4,chr1,1210602,1215800,RP5-902P8.10,;,chr1,1215686,1215687,AG,1,+,GGGCACTAGTGTACCT,1:1,17,chr1_1215687,acceptor_us_4_N,1
5,chr1,1210602,1215800,RP5-902P8.10,;,chr1,1215688,1215689,AG,1,-,GACTGCGAGGGCTTCC,1:1,17,chr1_1215689,acceptor_us_2_A,1
9,chr1,1407142,1433228,ATAD3B,;,chr1,1419947,1419948,AG,3,+,CGATGTATCCGGCACA,3:3,18,chr1_1419948,acceptor_us_1_G,1
11,chr1,1570602,1590473,CDK11B,;,chr1,1580934,1580935,TC,3,+,CGTCAGGTCCGTAGGC,3:3,17,chr1_1580935,acceptor_us_5_C_T,1
12,chr1,1570602,1590473,CDK11B,;,chr1,1580967,1580968,TC,3,+,CTACGTCAGTAGATGT,3:3,17,chr1_1580968,acceptor_us_4_N,1


In [273]:

judge_AS <- function(x){
    site_info=x[16]
    res_type=x[9]
    flag1 <- site_info == 'acceptor_us_1_G' & res_type=='AG'
    flag2 <- site_info == 'acceptor_us_2_A' & res_type=='AG'
    flag3 <- site_info == 'branch_site_A' & res_type=='AG'
    flag4 <- site_info == 'branch_site_A' & res_type=='AG'
    flag5 <- site_info == 'donor_ds_1_G' & res_type=='AG'
    flag6 <- site_info == 'donor_ds_2_T' & res_type=='TC'

    if(flag1 | flag2 | flag3 | flag4 | flag5 | flag6){
        return(x)
    }   
}


In [17]:
resInSpliceSites_WzGeneInfo_dedup_filtered <- c()
for(i in 1:nrow(resInSpliceSites_WzGeneInfo_dedup)){
    site_info=resInSpliceSites_WzGeneInfo_dedup[i,16]
    res_type=resInSpliceSites_WzGeneInfo_dedup[i,9]
    flag1 <- site_info == 'acceptor_us_1_G' & res_type=='AG'
    flag2 <- site_info == 'acceptor_us_2_A' & res_type=='AG'
    flag3 <- site_info == 'branch_site_A' & res_type=='AG'
    flag4 <- site_info == 'branch_site_A' & res_type=='AG'
    flag5 <- site_info == 'donor_ds_1_G' & res_type=='AG'
    flag6 <- site_info == 'donor_ds_2_T' & res_type=='TC'

    if(flag1 | flag2 | flag3 | flag4 | flag5 | flag6){
        resInSpliceSites_WzGeneInfo_dedup_filtered <- rbind(resInSpliceSites_WzGeneInfo_dedup_filtered,resInSpliceSites_WzGeneInfo_dedup[i,])
    }   
}



In [ ]:
#按中心法则，若该位点发生mRNA发生A-to-G，则DNA序列看到的应该是TC（正链），AG（负链）
resInSpliceSites_WzGeneInfo_dedup_filtered <- c()
for(i in 1:nrow(resInSpliceSites_WzGeneInfo_dedup)){
    site_info=resInSpliceSites_WzGeneInfo_dedup[i,16]
    res_type=resInSpliceSites_WzGeneInfo_dedup[i,9]
    flag1 <- site_info == 'acceptor_us_1_G' & res_type=='AG'
    flag2 <- site_info == 'acceptor_us_2_A' & res_type=='AG'
    flag3 <- site_info == 'branch_site_A' & res_type=='AG'
    flag4 <- site_info == 'branch_site_A' & res_type=='AG'
    flag5 <- site_info == 'donor_ds_1_G' & res_type=='AG'
    flag6 <- site_info == 'donor_ds_2_T' & res_type=='TC'

    if(flag1 | flag2 | flag3 | flag4 | flag5 | flag6){
        resInSpliceSites_WzGeneInfo_dedup_filtered <- rbind(resInSpliceSites_WzGeneInfo_dedup_filtered,resInSpliceSites_WzGeneInfo_dedup[i,])
    }   
}


In [18]:
dim(resInSpliceSites_WzGeneInfo_dedup_filtered)
#过滤后仍剩1273个基因，范围仍然很大
length(unique(resInSpliceSites_WzGeneInfo_dedup_filtered$V4))


[1] 1610   17

[1] 1273

In [21]:
#与有switch后果的基因做交叉
#得到'COPS7B''DNAJC10''ENTPD5''FAM193A''FRS2''MRI1''NPL''PALB2''TATDN3''TMEM198B''ZNF774'共11个基因
final_genes <- intersect(switch_cons$gene_id,unique(resInSpliceSites_WzGeneInfo_dedup$V4))
final_genes

[1] "COPS7B"   "DNAJC10"  "ENTPD5"   "FAM193A"  "FRS2"     "MRI1"    
 [7] "NPL"      "PALB2"    "TATDN3"   "TMEM198B" "ZNF774"

In [22]:
#isoform发生IR的基因与res发生的基因做交叉
genes_IR_RES <- intersect(names(table(deEx_iso_IR$gene_id)),unique(resInSpliceSites_WzGeneInfo_dedup$V4))
#最终候选研究基因取上述2种基因,得到12个候选基因
final_candidate_genes <- unique(c(final_genes,genes_IR_RES))
length(final_candidate_genes)
final_candidate_genes
#经研究背景调查，在12个candidate genes中得到经研究过与神经系统发育相关的6个基因
#head(resInSpliceSites_WzGeneInfo_dedup)
gene_name='DNAJC10'
resInSpliceSites_WzGeneInfo_dedup[resInSpliceSites_WzGeneInfo_dedup$V4 == gene_name,]

switch_cons[switch_cons$gene_id=='DNAJC10',]


[1] 12

[1] "COPS7B"   "DNAJC10"  "ENTPD5"   "FAM193A"  "FRS2"     "MRI1"    
 [7] "NPL"      "PALB2"    "TATDN3"   "TMEM198B" "ZNF774"   "MAN2B1"

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17
,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>
13102,chr2,183580998,183659191,DNAJC10,;,chr2,183587952,183587953,AG,1,+,CATCGAACAGTGGGAT,7:7,17,chr2_183587953,acceptor_us_2_A,1
13104,chr2,183580998,183659191,DNAJC10,;,chr2,183587519,183587520,AG,1,+,TGCTACCTCAGTCCCT,4:4,18,chr2_183587520,donor_ds_3_A_G,1
13117,chr2,183580998,183659191,DNAJC10,;,chr2,183588113,183588114,AG,1,-,GCATGCGAGTACGACG,1:1,23,chr2_183588114,donor_ds_4_A,1


,X,gene_ref,gene_id,gene_name,condition_1,condition_2,isoformUpregulated,isoformDownregulated,iso_ref_up,iso_ref_down,featureCompared,isoformsDifferent,switchConsequence
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>
81,81,geneComp_00006005,DNAJC10,DNAJC10,ASD,Control,ENST00000469118.5_1,ENST00000680480.1_1,isoComp_00027369,isoComp_00027379,intron_retention,TRUE,Intron retention loss
82,82,geneComp_00006005,DNAJC10,DNAJC10,ASD,Control,ENST00000469118.5_1,ENST00000680480.1_1,isoComp_00027369,isoComp_00027379,coding_potential,TRUE,Transcript is Noncoding
83,83,geneComp_00006005,DNAJC10,DNAJC10,ASD,Control,ENST00000469118.5_1,ENST00000680480.1_1,isoComp_00027369,isoComp_00027379,ORF_seq_similarity,TRUE,ORF is shorter
84,84,geneComp_00006005,DNAJC10,DNAJC10,ASD,Control,ENST00000469118.5_1,ENST00000680480.1_1,isoComp_00027369,isoComp_00027379,NMD_status,FALSE,NA
85,85,geneComp_00006005,DNAJC10,DNAJC10,ASD,Control,ENST00000469118.5_1,ENST00000680480.1_1,isoComp_00027369,isoComp_00027379,domains_identified,TRUE,Domain loss
86,86,geneComp_00006005,DNAJC10,DNAJC10,ASD,Control,ENST00000469118.5_1,ENST00000680648.1_1,isoComp_00027369,isoComp_00027381,intron_retention,FALSE,NA
87,87,geneComp_00006005,DNAJC10,DNAJC10,ASD,Control,ENST00000469118.5_1,ENST00000680648.1_1,isoComp_00027369,isoComp_00027381,coding_potential,FALSE,NA
88,88,geneComp_00006005,DNAJC10,DNAJC10,ASD,Control,ENST00000469118.5_1,ENST00000680648.1_1,isoComp_00027369,isoComp_00027381,ORF_seq_similarity,TRUE,ORF is shorter
89,89,geneComp_00006005,DNAJC10,DNAJC10,ASD,Control,ENST00000469118.5_1,ENST00000680648.1_1,isoComp_00027369,isoComp_00027381,NMD_status,FALSE,NA


In [38]:
gene_name='ENTPD5'
resInSpliceSites_WzGeneInfo_dedup[resInSpliceSites_WzGeneInfo_dedup$V4 == gene_name,]


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17
,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>
5849,chr14,74424712,74486102,ENTPD5,;,chr14,74463480,74463481,TC,5,-,GACGGCTCAAAGTCAA,7:7,18,chr14_74463481,donor_ds_2_T,1


In [ ]:
resInSpliceSites_WzGeneInfo_dedup[final_candidate_genes]
resInSpliceSites_WzGeneInfo_dedup_filtered <- apply(resInSpliceSites_WzGeneInfo_dedup,1,judge_AS)

dim(resInSpliceSites_WzGeneInfo_dedup)


In [274]:
resInSpliceSites_WzGeneInfo_dedup_filtered <- apply(resInSpliceSites_WzGeneInfo_dedup,1,judge_AS)


In [276]:
dim(resInSpliceSites_WzGeneInfo_dedup)

[1] 4306   17

In [257]:

#做功能富集分析，以此为基础选定后续分析使用的基因
gene_names <- names(table(resInSpliceSites_WzGeneInfo_dedup$V4))
gene_id <- mget(gene_names, #需要转换的Symbol
                     org.Hs.egSYMBOL2EG, # Symbol转EntrezID（人）
                     ifnotfound=NA)
gene_id <- as.character(gene_id)[!is.na(gene_id)]


In [258]:
length(gene_id)

[1] 1836

In [259]:

enrich.go <- enrichGO(gene = gene_id,  
                      OrgDb = 'org.Hs.eg.db',  
                      keyType = 'ENTREZID',  #指定给定的基因名称类型，例如这里以 entrze id 为例
                      ont = 'ALL',  #GO Ontology，可选 BP、MF、CC，也可以指定 ALL 同时计算 3 者
                      pAdjustMethod = 'fdr',  #指定 p 值校正方法
                      pvalueCutoff = 1,  #指定 p 值阈值（可指定 1 以输出全部）
                      qvalueCutoff = 1,  #指定 q 值阈值（可指定 1 以输出全部）
                      readable = FALSE)  



In [261]:
#####
pdf("/disk1/wenqing/tmp_data/ASD/GO_enrichment_genesWzResInAsSite.pdf")
dotplot(enrich.go)  #富集气泡图
barplot(enrich.go)  #富集柱形图
dev.off()

png 
  2

In [ ]:

write.table(input_genes,"/disk1/wenqing/tmp_data/ASD/gene_names_GO.txt",quote=F,row.names=F)

In [8]:
#587个iso发生可变剪接的形式（具体位点）
AlterSplice <- read.csv("/disk1/wenqing/tmp_data/ASD/AlternativeSplicingAnalysis.csv")
head(AlterSplice)


,X,isoform_id,ES,ES_genomic_start,ES_genomic_end,MEE,MEE_genomic_start,MEE_genomic_end,MES,MES_genomic_start,⋯,A5_genomic_end,A3,A3_genomic_start,A3_genomic_end,ATSS,ATSS_genomic_start,ATSS_genomic_end,ATTS,ATTS_genomic_start,ATTS_genomic_end
,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,ENST00000648157.1_1,0,NA,NA,0,NA,NA,0,NA,⋯,NA,0,NA,NA,1,9217759,9217798,1,9221280,9221285
2,2,ENST00000667950.1_2,0,NA,NA,0,NA,NA,0,NA,⋯,NA,0,NA,NA,0,NA,NA,0,NA,NA
3,3,ENST00000670582.1_1,0,NA,NA,0,NA,NA,0,NA,⋯,NA,0,NA,NA,1,9217759,9217770,1,9221258,9221285
4,4,ENST00000357096.5_4,1,131347198,131347351,0,NA,NA,1,131310451;131310586,⋯,131321100;131329829,2,131296314;131309051,131296638;131309093,0,NA,NA,1,131285666,131285669
5,5,ENST00000489047.1_3,0,NA,NA,0,NA,NA,0,NA,⋯,NA,0,NA,NA,1,131323783;131324498;131325100;131325793;131326546;131329724;131347198;131347463,131323919;131324597;131325201;131325857;131326660;131329944;131347351;131347607,1,131285666;131295229;131296213;131298228;131302109;131303633;131305821;131307243;131308417;131308959;131310451;131310586;131312341,131290064;131295300;131296638;131298371;131302225;131303721;131305893;131307338;131308551;131309093;131310528;131310663;131312414
6,6,ENST00000492156.1_3,0,NA,NA,0,NA,NA,0,NA,⋯,NA,0,NA,NA,1,131298228;131302109;131303633;131305821;131307243;131308417;131308959;131310451;131310586;131312341;131317552;131322515;131323741;131324498;131325100;131325793;131326546;131329724;131347198;131347463,131298371;131302225;131303721;131305893;131307338;131308551;131309093;131310528;131310663;131312414;131321151;131322547;131323919;131324597;131325201;131325857;131326660;131329944;131347351;131347607,1,131285666;131295229,131290064;131295266


In [20]:
#usage发生变化的iso的后果（94个基因）
switch_cons <- read.csv("/disk1/wenqing/tmp_data/ASD/switchConsequence.csv")
#isoform在每个样本的表达信息，用于决定导入哪些样本的res
iso_abundance_filtered <- read.csv("/disk1/wenqing/tmp_data/ASD/isoformEx_abundance_filtered.csv")
rownames(iso_abundance_filtered) <- iso_abundance_filtered$isoform_id


In [111]:
head(switch_cons)
switch_cons[switch_cons$gene_id=='USP34-DT',]
write.table(unique(switch_cons$gene_id),"/disk1/wenqing/tmp_data/ASD/geneWzAS.txt",quote=F,row.names=F)

,X,gene_ref,gene_id,gene_name,condition_1,condition_2,isoformUpregulated,isoformDownregulated,iso_ref_up,iso_ref_down,featureCompared,isoformsDifferent,switchConsequence
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>
1,1,geneComp_00000007,A2M-AS1,A2M-AS1,ASD,Control,ENST00000648157.1_1,ENST00000670582.1_1,isoComp_00000023,isoComp_00000025,intron_retention,TRUE,Intron retention loss
2,2,geneComp_00000007,A2M-AS1,A2M-AS1,ASD,Control,ENST00000648157.1_1,ENST00000670582.1_1,isoComp_00000023,isoComp_00000025,coding_potential,FALSE,NA
3,3,geneComp_00000007,A2M-AS1,A2M-AS1,ASD,Control,ENST00000648157.1_1,ENST00000670582.1_1,isoComp_00000023,isoComp_00000025,ORF_seq_similarity,TRUE,ORF is shorter
4,4,geneComp_00000007,A2M-AS1,A2M-AS1,ASD,Control,ENST00000648157.1_1,ENST00000670582.1_1,isoComp_00000023,isoComp_00000025,NMD_status,TRUE,NMD sensitive
5,5,geneComp_00000007,A2M-AS1,A2M-AS1,ASD,Control,ENST00000648157.1_1,ENST00000670582.1_1,isoComp_00000023,isoComp_00000025,domains_identified,FALSE,NA
6,6,geneComp_00000503,ACSL6,ACSL6,ASD,Control,ENST00000357096.5_4,ENST00000651883.2_8,isoComp_00001614,isoComp_00001624,intron_retention,FALSE,NA


,X,gene_ref,gene_id,gene_name,condition_1,condition_2,isoformUpregulated,isoformDownregulated,iso_ref_up,iso_ref_down,featureCompared,isoformsDifferent,switchConsequence
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>
531,531,geneComp_00040665,USP34-DT,USP34-DT,ASD,Control,ENST00000603652.2_4,ENST00000702649.1_1,isoComp_00133928,isoComp_00133932,intron_retention,TRUE,Intron retention switch
532,532,geneComp_00040665,USP34-DT,USP34-DT,ASD,Control,ENST00000603652.2_4,ENST00000702649.1_1,isoComp_00133928,isoComp_00133932,coding_potential,FALSE,NA
533,533,geneComp_00040665,USP34-DT,USP34-DT,ASD,Control,ENST00000603652.2_4,ENST00000702649.1_1,isoComp_00133928,isoComp_00133932,ORF_seq_similarity,TRUE,Complete ORF gain
534,534,geneComp_00040665,USP34-DT,USP34-DT,ASD,Control,ENST00000603652.2_4,ENST00000702649.1_1,isoComp_00133928,isoComp_00133932,NMD_status,NA,NA
535,535,geneComp_00040665,USP34-DT,USP34-DT,ASD,Control,ENST00000603652.2_4,ENST00000702649.1_1,isoComp_00133928,isoComp_00133932,domains_identified,FALSE,NA
536,536,geneComp_00040665,USP34-DT,USP34-DT,ASD,Control,ENST00000605437.1_2,ENST00000702649.1_1,isoComp_00133929,isoComp_00133932,intron_retention,TRUE,Intron retention loss
537,537,geneComp_00040665,USP34-DT,USP34-DT,ASD,Control,ENST00000605437.1_2,ENST00000702649.1_1,isoComp_00133929,isoComp_00133932,coding_potential,FALSE,NA
538,538,geneComp_00040665,USP34-DT,USP34-DT,ASD,Control,ENST00000605437.1_2,ENST00000702649.1_1,isoComp_00133929,isoComp_00133932,ORF_seq_similarity,TRUE,Complete ORF gain
539,539,geneComp_00040665,USP34-DT,USP34-DT,ASD,Control,ENST00000605437.1_2,ENST00000702649.1_1,isoComp_00133929,isoComp_00133932,NMD_status,NA,NA


In [132]:
dim(switch_cons)
#switch_cons_rm <- switch_cons[!is.na(switch_cons$switchConsequence),]
#dim(switch_cons_rm)
#head(switch_cons_rm)

[1] 580  13

[1] 243  13

,X,gene_ref,gene_id,gene_name,condition_1,condition_2,isoformUpregulated,isoformDownregulated,iso_ref_up,iso_ref_down,featureCompared,isoformsDifferent,switchConsequence
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>
1,1,geneComp_00000007,A2M-AS1,A2M-AS1,ASD,Control,ENST00000648157.1_1,ENST00000670582.1_1,isoComp_00000023,isoComp_00000025,intron_retention,TRUE,Intron retention loss
3,3,geneComp_00000007,A2M-AS1,A2M-AS1,ASD,Control,ENST00000648157.1_1,ENST00000670582.1_1,isoComp_00000023,isoComp_00000025,ORF_seq_similarity,TRUE,ORF is shorter
4,4,geneComp_00000007,A2M-AS1,A2M-AS1,ASD,Control,ENST00000648157.1_1,ENST00000670582.1_1,isoComp_00000023,isoComp_00000025,NMD_status,TRUE,NMD sensitive
8,8,geneComp_00000503,ACSL6,ACSL6,ASD,Control,ENST00000357096.5_4,ENST00000651883.2_8,isoComp_00001614,isoComp_00001624,ORF_seq_similarity,TRUE,ORF is shorter
10,10,geneComp_00000503,ACSL6,ACSL6,ASD,Control,ENST00000357096.5_4,ENST00000651883.2_8,isoComp_00001614,isoComp_00001624,domains_identified,TRUE,Domain gain
12,12,geneComp_00001500,APBB2,APBB2,ASD,Control,ENST00000508707.5_4,ENST00000295974.12_6,isoComp_00005871,isoComp_00005861,coding_potential,TRUE,Transcript is Noncoding


In [5]:
# dim(deEx_iso)
#deEx_switch_consequence_iso_id <- intersect(deEx_iso$isoform_id,c(switch_cons$isoformUpregulated,switch_cons$isoformDownregulated))


In [11]:
#取有明确功能后果的iso进行分析
deEx_iso_IR_range <- exon_region[exon_region$isoform_id %in% deEx_iso_IR$isoform_id,]

In [15]:
head(deEx_iso_IR_range)
dim(deEx_iso_IR_range)
length(unique(deEx_iso_IR_range$isoform_id))


,X,seqnames,start,end,width,strand,isoform_id,gene_id,gene_name
,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
76395,76395,chr1,161952982,161954066,1085,-,ENST00000294794.8_9,OLFML2B,OLFML2B
76398,76398,chr1,161954594,161954770,177,-,ENST00000294794.8_9,OLFML2B,OLFML2B
76399,76399,chr1,161967615,161968139,525,-,ENST00000294794.8_9,OLFML2B,OLFML2B
76400,76400,chr1,161969903,161970128,226,-,ENST00000294794.8_9,OLFML2B,OLFML2B
76401,76401,chr1,161976087,161976263,177,-,ENST00000294794.8_9,OLFML2B,OLFML2B
76402,76402,chr1,161987190,161987297,108,-,ENST00000294794.8_9,OLFML2B,OLFML2B


[1] 376   9

[1] 71

In [16]:
#构建总的res列表
res_17 <- read.table("/disk1/wenqing/tmp_data/ASD/asd_male_pfc/asd_17/SPRINT_identified_all.res")
res_18 <- read.table("/disk1/wenqing/tmp_data/ASD/asd_male_pfc/asd_18/SPRINT_identified_all.res")
res_57 <- read.table("/disk1/wenqing/tmp_data/ASD/asd_male_pfc/asd_57/SPRINT_identified_all.res")
res_20 <- read.table("/disk1/wenqing/tmp_data/ASD/ctr_male_pfc/ctr_20/SPRINT_identified_all.res")
res_23 <- read.table("/disk1/wenqing/tmp_data/ASD/ctr_male_pfc/ctr_23/SPRINT_identified_all.res")
res_56 <- read.table("/disk1/wenqing/tmp_data/ASD/ctr_male_pfc/ctr_56/SPRINT_identified_all.res")

all_res <- list()
all_res[['17']] <- res_17
all_res[['18']] <- res_18
all_res[['57']] <- res_57
all_res[['20']] <- res_20
all_res[['23']] <- res_23
all_res[['56']] <- res_56



In [63]:
dim(unlist(all_res))

NULL

In [34]:
#####构建剪切供受体位点
start_end_site <- c(paste0(deEx_iso_IR_range$seqnames,"_",deEx_iso_IR_range$start),paste0(deEx_iso_IR_range$seqnames,"_",deEx_iso_IR_range$end))
#all_site <- c(start_site_expand,end_site_expand,start_end_site)
deEx_iso_IR_id <- rep(deEx_iso_IR_range$isoform_id,times=2)
deEx_iso_sites <- as.data.frame(cbind(start_end_site,deEx_iso_IR_id))
colnames(deEx_iso_sites) <- c('site','iso_id')
head(deEx_iso_sites)

#deEx_iso_sites <- deEx_iso_sites[!duplicated(deEx_iso_sites$site),]
#dim(deEx_iso_sites)
#deEx_iso_sites[deEx_iso_sites$iso_id=='ENST00000294794.8_9',]

#对每一个iso进行遍历，选择做交叉的res
###未找到位于剪接供受体位置的res
for(one_iso_id in deEx_iso_sites$iso_id){
  loc <- which(iso_abundance_filtered[one_iso_id,c(3:8)]!=0)
  input_res <- all_res[[loc[1]]]
  for(i in loc[-1]){
    tmp_input_res <- all_res[[i]]
    input_res <- rbind(input_res,tmp_input_res)
  }
  all_res_site <- paste0(input_res$V1,"_",input_res$V3)
  resInIsoSite <- intersect(deEx_iso_sites$site,all_res_site)
  if(length(resInIsoSite)!=0){
    print(one_iso_id)
  }
}

In [67]:
###找位于iso边界上下游5k region的res
start <- deEx_iso_IR_range$start-5000
end <- deEx_iso_IR_range$end+5000
new_region <- cbind(deEx_iso_IR_range$seqnames,start,end,deEx_iso_IR_range$isoform_id,deEx_iso_IR_range$gene_id)

dim(new_region)
colnames(new_region) <- NULL


[1] 376   5

,start,end,,
chr1,161947982,161959066,ENST00000294794.8_9,OLFML2B
chr1,161949594,161959770,ENST00000294794.8_9,OLFML2B
chr1,161962615,161973139,ENST00000294794.8_9,OLFML2B
chr1,161964903,161975128,ENST00000294794.8_9,OLFML2B
chr1,161971087,161981263,ENST00000294794.8_9,OLFML2B
chr1,161982190,161992297,ENST00000294794.8_9,OLFML2B


In [127]:

head(new_region)
length(unique(deEx_iso_IR_range$gene_id))
table(deEx_iso_IR_range$gene_id)

chr1,161947982,161959066,ENST00000294794.8_9,OLFML2B
chr1,161949594,161959770,ENST00000294794.8_9,OLFML2B
chr1,161962615,161973139,ENST00000294794.8_9,OLFML2B
chr1,161964903,161975128,ENST00000294794.8_9,OLFML2B
chr1,161971087,161981263,ENST00000294794.8_9,OLFML2B
chr1,161982190,161992297,ENST00000294794.8_9,OLFML2B


[1] 39


        A2M-AS1           APBB2          ARMCX6        C18orf21           CNDP2 
              4               3               3               4               7 
         COPS7B           CYREN          DCAF11          DCAF17         DNAJC10 
             12               4              34              14              53 
ENSG00000287625 ENSG00000291171         FAM193A         FAM200B          FIGNL1 
              7               8               2              18               2 
        GAS6-DT         GLUD1P3             HGF          HS1BP3           IFIT5 
              4               1               4               2               2 
        KRTCAP3           LERFS       LINC03015      LYPLAL1-DT          MAN2B1 
              3               2               5               5               2 
         MIATNB         OLFML2B           PALB2            PAX6            PCTP 
              2               8              12              33               6 
           PEG3          PN

In [68]:
write.table(new_region,"/disk1/wenqing/tmp_data/ASD/region.txt",quote=F,row.names=F)

In [93]:
sample <- c(rep('17',times=2777043),rep('18',times=5445279),rep('57',times=2130754),rep('20',times=3506164),rep('23',times=4297423),rep('26',times=3721936))
length(sample)

[1] 21878599

In [91]:
write.table(as.data.frame(sample),"/disk1/wenqing/tmp_data/ASD/sample_info.txt",quote=F,row.names=F)

In [94]:
######得到位于region区域的res——AsdCtrResInIsoformRegion.txt
resInRegion <- read.table("/disk1/wenqing/tmp_data/ASD/AsdCtrResInIsoformRegion.txt")
head(resInRegion)
table(resInRegion$V5)

#生成isoform_res唯一标识符
res_iso_uniq_id <- paste0(resInRegion$V4,"_",resInRegion$V6,"_",resInRegion$V8)
length(res_iso_uniq_id)
#共8062个res位于isoform上下游5k的范围内
length(unique(res_iso_uniq_id))

resInRegion_uniq <- resInRegion[!duplicated(res_iso_uniq_id),]
write.table(resInRegion_uniq[resInRegion_uniq$V5=='USP34-DT',],"/disk1/wenqing/tmp_data/ASD/USP34DT_res.txt",quote=F,row.names=F)
sort(table(resInRegion[!duplicated(res_iso_uniq_id),]$V5),decreasing=T)

length(names(sort(table(resInRegion[!duplicated(res_iso_uniq_id),]$V5),decreasing=T)))


In [106]:
#以USP34-DT长非编码RNA为例，查看其isoform与res位置的关系
head(exon_region)
exon_region[exon_region$isoform_id=='ENST00000603652.2_4',]
exon_region[exon_region$isoform_id=='ENST00000702649.1_1',]

,X,seqnames,start,end,width,strand,isoform_id,gene_id,gene_name
,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
1,1,chr1,12010,12057,48,+,ENST00000450305.2_2,DDX11L1,DDX11L1
2,2,chr1,12179,12227,49,+,ENST00000450305.2_2,DDX11L1,DDX11L1
3,3,chr1,12613,12697,85,+,ENST00000450305.2_2,DDX11L1,DDX11L1
4,4,chr1,12975,13052,78,+,ENST00000450305.2_2,DDX11L1,DDX11L1
5,5,chr1,13221,13374,154,+,ENST00000450305.2_2,DDX11L1,DDX11L1
6,6,chr1,13453,13670,218,+,ENST00000450305.2_2,DDX11L1,DDX11L1


,X,seqnames,start,end,width,strand,isoform_id,gene_id,gene_name
,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
96911,96911,chr2,61698323,61698514,192,+,ENST00000603652.2_4,USP34-DT,USP34-DT
96919,96919,chr2,61707460,61707718,259,+,ENST00000603652.2_4,USP34-DT,USP34-DT
96922,96922,chr2,61709762,61711048,1287,+,ENST00000603652.2_4,USP34-DT,USP34-DT


,X,seqnames,start,end,width,strand,isoform_id,gene_id,gene_name
,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
96915,96915,chr2,61698521,61699215,695,+,ENST00000702649.1_1,USP34-DT,USP34-DT


In [ ]:
#################方案2:除去经选择性剪接分析得到的71个具有IR的iso外，对94个经历了选择性剪接且有明确后果的基因进行igv的详细可视化
#1.获取94个基因的region
gene_bed <- read.table("/disk1/wenqing/tmp_data/gene.bed")
head(gene_bed)

length(unique(switch_cons$gene_id))
setdiff(unique(switch_cons$gene_id),gene_bed$V4)
table(gene_bed$V4 %in% unique(switch_cons$gene_id))
geneWzAS_region <- gene_bed[gene_bed$V4 %in% unique(switch_cons$gene_id),]
head(geneWzAS_region)
write.table(geneWzAS_region,"/disk1/wenqing/tmp_data/ASD/geneWzAS_region.txt",quote=F,row.names=F)

In [156]:
#2.相较于isoform，不针对基因的上下游做扩展
resInGeneRegion <- read.table("/disk1/wenqing/tmp_data/ASD/AsdCtrResInGeneRegion.txt")
head(resInGeneRegion)
table(resInGeneRegion$V4)


,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14
,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>
1,chr1,23707553,23751233,TCEA3,chr1,23708804,23708805,TC,2,+,CTCGTCATCGCGCCAA,2:2,17,1
2,chr1,23707553,23751233,TCEA3,chr1,23708805,23708806,TC,2,+,CTCGTCATCGCGCCAA,2:2,17,1
3,chr1,23707553,23751233,TCEA3,chr1,23708821,23708822,TC,2,+,CTCGTCATCGCGCCAA,2:2,17,1
4,chr1,23707553,23751233,TCEA3,chr1,23718170,23718171,TC,1,+,AGTGGGAGTTCACGGC,1:1,17,1
5,chr1,23707553,23751233,TCEA3,chr1,23718171,23718172,TC,1,+,AGTGGGAGTTCACGGC,1:1,17,1
6,chr1,23707553,23751233,TCEA3,chr1,23718172,23718173,TC,1,+,AGTGGGAGTTCACGGC,1:1,17,1



          ACSL6           APBB2           ARMC2           ATMIN             CAT 
           1232            7189            1582              13             109 
           CAV1            CDH1            CLP1           CNDP2          COPS7B 
             85             664             134              44              98 
          CYREN          DCAF11          DCAF17         DNAJC10           DPP10 
            198              74             977             422           38635 
            EDA        EDRF1-DT           EGFL8 ENSG00000228395 ENSG00000228863 
            644              14              50             249               8 
ENSG00000249335 ENSG00000286329 ENSG00000291054 ENSG00000291138 ENSG00000291171 
           1077              81            1163             121             651 
         ENTPD5           EPRS1         FAM193A         FAM200B          FIGNL1 
            607             923            4895              13               3 
           FKRP            

In [157]:

#生成isoform_res唯一标识符
res_iso_uniq_id <- paste0(resInGeneRegion$V4,"_",resInGeneRegion$V5,"_",resInGeneRegion$V7)
head(res_iso_uniq_id)
#共8062个res位于isoform上下游5k的范围内
length(unique(res_iso_uniq_id))


[1] "TCEA3_chr1_23708805" "TCEA3_chr1_23708806" "TCEA3_chr1_23708822"
[4] "TCEA3_chr1_23718171" "TCEA3_chr1_23718172" "TCEA3_chr1_23718173"

[1] 38132

In [158]:

resInGeneRegion_uniq <- resInGeneRegion[!duplicated(res_iso_uniq_id),]


In [197]:
head(resInGeneRegion_uniq)
####85个具有AS事件的基因显示如下数量的RES
length(table(resInGeneRegion_uniq$V4))
table(resInGeneRegion_uniq$V4)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14
,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>
1,chr1,23707553,23751233,TCEA3,chr1,23708804,23708805,TC,2,+,CTCGTCATCGCGCCAA,2:2,17,1
2,chr1,23707553,23751233,TCEA3,chr1,23708805,23708806,TC,2,+,CTCGTCATCGCGCCAA,2:2,17,1
3,chr1,23707553,23751233,TCEA3,chr1,23708821,23708822,TC,2,+,CTCGTCATCGCGCCAA,2:2,17,1
4,chr1,23707553,23751233,TCEA3,chr1,23718170,23718171,TC,1,+,AGTGGGAGTTCACGGC,1:1,17,1
5,chr1,23707553,23751233,TCEA3,chr1,23718171,23718172,TC,1,+,AGTGGGAGTTCACGGC,1:1,17,1
6,chr1,23707553,23751233,TCEA3,chr1,23718172,23718173,TC,1,+,AGTGGGAGTTCACGGC,1:1,17,1


[1] 85


          ACSL6           APBB2           ARMC2           ATMIN             CAT 
            595            2748             804              13              54 
           CAV1            CDH1            CLP1           CNDP2          COPS7B 
             46             387              71              24              71 
          CYREN          DCAF11          DCAF17         DNAJC10           DPP10 
            147              58             388             135            7769 
            EDA        EDRF1-DT           EGFL8 ENSG00000228395 ENSG00000228863 
            476              12              25             175               8 
ENSG00000249335 ENSG00000286329 ENSG00000291054 ENSG00000291138 ENSG00000291171 
            595              59             670              39             323 
         ENTPD5           EPRS1         FAM193A         FAM200B          FIGNL1 
            417             478            1770              13               3 
           FKRP            

In [203]:
#在17这个样本中，发生在TCF7L2基因上的res共43个
resInGeneRegion_uniq[resInGeneRegion_uniq$V4=='TCF7L2' & resInGeneRegion_uniq$V13=='17',]
###后续两种方向：
#1:每一簇的res周围序列的情况做分析，是否与剪接供受体序列特点一致，
#2.ASD/Ctr样本的res有什么不同，这种不同对可变剪接有什么贡献

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14
,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<int>,<int>
3994,chr10,114710008,114927437,TCF7L2,chr10,114725722,114725723,AG,1,-,GCGCCAACACACGCTG,1:1,17,1
3995,chr10,114710008,114927437,TCF7L2,chr10,114725731,114725732,AG,1,-,GCGCCAACACACGCTG,1:1,17,1
3996,chr10,114710008,114927437,TCF7L2,chr10,114725745,114725746,AG,1,-,CGGGTCAAGGGAGTAA,1:1,17,1
3997,chr10,114710008,114927437,TCF7L2,chr10,114725746,114725747,AG,1,-,CGGGTCAAGGGAGTAA,1:1,17,1
3998,chr10,114710008,114927437,TCF7L2,chr10,114725748,114725749,AG,3,-,AACGTTGAGGGTTCCC,1:1,17,1
4139,chr10,114710008,114927437,TCF7L2,chr10,114772957,114772958,AG,1,+,GTCACGGTCTCTAAGG,1:1,17,1
4140,chr10,114710008,114927437,TCF7L2,chr10,114772970,114772971,AG,1,+,GTCACGGTCTCTAAGG,2:2,17,1
4141,chr10,114710008,114927437,TCF7L2,chr10,114772994,114772995,AG,1,+,GACTGCGAGTTTCCTT,1:1,17,1
4142,chr10,114710008,114927437,TCF7L2,chr10,114773004,114773005,AG,1,+,GTATCTTAGGTGCAAC,2:2,17,1


In [160]:
write.table(resInGeneRegion_uniq[resInGeneRegion_uniq$V4=='TCF7L2',],"/disk1/wenqing/tmp_data/ASD/TCF7L2_res.txt",quote=F,row.names=F)



In [168]:
#基因的region过大，以基因上的usage有差异的isoform为锚点
#head(exon_region)
exon_region[exon_region$isoform_id=='ENST00000534894.5_4',]
114725723-114710704
114900943-114835735

,X,seqnames,start,end,width,strand,isoform_id,gene_id,gene_name
,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>
483922,483922,chr10,114710009,114710704,696,+,ENST00000534894.5_4,TCF7L2,TCF7L2
483926,483926,chr10,114710966,114711032,67,+,ENST00000534894.5_4,TCF7L2,TCF7L2
483930,483930,chr10,114711242,114711366,125,+,ENST00000534894.5_4,TCF7L2,TCF7L2
483949,483949,chr10,114900943,114901075,133,+,ENST00000534894.5_4,TCF7L2,TCF7L2
483962,483962,chr10,114903682,114903784,103,+,ENST00000534894.5_4,TCF7L2,TCF7L2
483973,483973,chr10,114905770,114905856,87,+,ENST00000534894.5_4,TCF7L2,TCF7L2
483987,483987,chr10,114910757,114910882,126,+,ENST00000534894.5_4,TCF7L2,TCF7L2
483998,483998,chr10,114911484,114911643,160,+,ENST00000534894.5_4,TCF7L2,TCF7L2
484008,484008,chr10,114912092,114912199,108,+,ENST00000534894.5_4,TCF7L2,TCF7L2


[1] 15019

[1] 65208

In [169]:
switch_cons[switch_cons$gene_id=='TCF7L2',]
80*47

,X,gene_ref,gene_id,gene_name,condition_1,condition_2,isoformUpregulated,isoformDownregulated,iso_ref_up,iso_ref_down,featureCompared,isoformsDifferent,switchConsequence
,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<lgl>,<chr>
496,496,geneComp_00038690,TCF7L2,TCF7L2,ASD,Control,ENST00000704415.1_1,ENST00000534894.5_4,isoComp_00125208,isoComp_00125203,intron_retention,FALSE,NA
497,497,geneComp_00038690,TCF7L2,TCF7L2,ASD,Control,ENST00000704415.1_1,ENST00000534894.5_4,isoComp_00125208,isoComp_00125203,coding_potential,TRUE,Transcript is Noncoding
498,498,geneComp_00038690,TCF7L2,TCF7L2,ASD,Control,ENST00000704415.1_1,ENST00000534894.5_4,isoComp_00125208,isoComp_00125203,ORF_seq_similarity,TRUE,ORF is shorter
499,499,geneComp_00038690,TCF7L2,TCF7L2,ASD,Control,ENST00000704415.1_1,ENST00000534894.5_4,isoComp_00125208,isoComp_00125203,NMD_status,FALSE,NA
500,500,geneComp_00038690,TCF7L2,TCF7L2,ASD,Control,ENST00000704415.1_1,ENST00000534894.5_4,isoComp_00125208,isoComp_00125203,domains_identified,TRUE,Domain loss


[1] 3760

In [181]:
#####对有明确switch后果的基因做功能富集分析
library("clusterProfiler")
library("org.Hs.eg.db")
library(repr)

In [191]:
input_genes <- unique(switch_cons$gene_id)
head(input_genes)

[1] "A2M-AS1" "ACSL6"   "APBB2"   "ARMC2"   "ARMCX6"  "ATMIN"

In [177]:

gene_id <- mget(input_genes, #需要转换的Symbol
                     org.Hs.egSYMBOL2EG, # Symbol转EntrezID（人）
                     ifnotfound=NA)
gene_id <- as.character(gene_id)[!is.na(gene_id)]

length(gene_id)

enrich.go


enrich.go <- enrichGO(gene = gene_id,  
                      OrgDb = 'org.Hs.eg.db',  
                      keyType = 'ENTREZID',  #指定给定的基因名称类型，例如这里以 entrze id 为例
                      ont = 'ALL',  #GO Ontology，可选 BP、MF、CC，也可以指定 ALL 同时计算 3 者
                      pAdjustMethod = 'fdr',  #指定 p 值校正方法
                      pvalueCutoff = 1,  #指定 p 值阈值（可指定 1 以输出全部）
                      qvalueCutoff = 1,  #指定 q 值阈值（可指定 1 以输出全部）
                      readable = FALSE)  


write.table(input_genes,"/disk1/wenqing/tmp_data/ASD/gene_names_GO.txt",quote=F,row.names=F)

[1] 87

integer(0)

In [195]:
#####enrich.go为空，所得94个基因未富集到显著功能
pdf("/disk1/wenqing/tmp_data/ASD/GO_enrichment_switchGenes.pdf")
dotplot(enrich.go)  #富集气泡图
barplot(enrich.go)  #富集柱形图
dev.off()

png 
  2

In [209]:
114927437-114710008

114710008+441246

[1] 217429

In [2]:
library(Seurat)

Attaching SeuratObject



In [3]:
library(BiocManager)

In [4]:
BiocManager::install("batchelor")

'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.17 (BiocManager 1.30.21.1), R 4.3.1 (2023-06-16)

Installing package(s) 'batchelor'

also installing the dependencies ‘RcppHNSW’, ‘rsvd’, ‘DelayedArray’, ‘sparseMatrixStats’, ‘SingleCellExperiment’, ‘BiocNeighbors’, ‘BiocSingular’, ‘DelayedMatrixStats’, ‘scuttle’, ‘ResidualMatrix’, ‘ScaledMatrix’, ‘beachmat’


Warning message in install.packages(...):
“installation of package ‘RcppHNSW’ had non-zero exit status”
Warning message in install.packages(...):
“installation of package ‘DelayedArray’ had non-zero exit status”
Warning message in install.packages(...):
“installation of package ‘sparseMatrixStats’ had non-zero exit status”
Warning message in install.packages(...):
“installation of package ‘BiocNeighbors’ had non-zero exit status”
Warning message in install.packages(

In [9]:
116422296*100

7013391300+4628838258

[1] 11642229600

[1] 11642229558